---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    graphs = {}
    graphs_degrees = {}
    for i in range(len(P1_Graphs)):
        graphs["G{}".format(i+1)] = P1_Graphs[i]
        graphs_degrees["G{}".format(i+1)] = P1_Graphs[i].degree()
        
    #loop through P1 graphs and create a dictionary with names, also create a dictionary with 
    #the degree of each graph
             
    graphs_degrees_values = {}
    for i, graph in enumerate(graphs_degrees):
        graphs_degrees_values["G{}".format(i+1)] = set(graphs_degrees[graph].values())
        print()
    
    histograms = {}
    for i, graph in enumerate(graphs_degrees_values):
        histogram_values = []
        for j, degree in enumerate(graphs_degrees_values[graph]):
            histogram_values.append(list(graphs_degrees[graph].values()).count(degree)/nx.number_of_nodes(graphs[graph]))
        histograms["G{}".format(i+1)] = histogram_values

    
    #fig, axs = plt.subplots(3,2, sharex=True, sharey=True)
    counter = 0
    for i in range(3):
        for j in range(2):
            counter +=1
            #axs[i][j].set_xscale('log')
            #axs[i][j].set_yscale('log')
            try:
                pass
                #axs[i][j].plot(list(graphs_degrees_values["G{}".format(counter)]),histograms["G{}".format(counter)])
            except:
                pass
                
    
    answer = ['PA','SW_L','SW_L','PA','SW_H']
        
    return answer

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV

def salary_predictions():
    
    department = list(nx.get_node_attributes(G,'Department').values())
    management_salary = list(nx.get_node_attributes(G,'ManagementSalary').values())
    index = list(nx.get_node_attributes(G,'Department').keys())
    clustering = list(nx.clustering(G).values())
    degree = list(G.degree().values())
    
    df = pd.DataFrame(index = index)
    df['Department'] = department
    df['Clustering'] = clustering
    df['Degree'] = degree
    df['PageRank'] = nx.pagerank(G,alpha=0.85).values()
    #df['HubScore'] = nx.hits(G)[0]
    #df['AuthScore'] = nx.hits(G)[1]
    df['ManagementSalary'] = management_salary

    hold_out_data = df.loc[df['ManagementSalary'].isnull()]
    training_data = df.loc[~df['ManagementSalary'].isnull()]
    X_hold_out = hold_out_data.iloc[:,0:-1]
 
    
    X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:,0:-1],
                                                        training_data['ManagementSalary'], 
                                                        test_size = 0.25, random_state = 1)
    
    #clf = KNeighborsClassifier(n_neighbors=3).fit(X_train,y_train)
    #clf = DecisionTreeClassifier().fit(X_train,y_train)
    clf = LogisticRegression(solver = 'liblinear', C=1,penalty='l1').fit(X_train,y_train)
    #clf = LinearSVC(C=0.1).fit(X_train,y_train)
    #clf = SVC(C=1,gamma=0.001).fit(X_train,y_train)
    y_test_predicted = clf.decision_function(X_test)
    y_hold_out_predicted = clf.predict_proba(X_hold_out)

    
    param_grid = {'C':[0.001,0.01,0.1,1,10,100],'penalty':['l1','l2']}
    grid_search = GridSearchCV(clf,
                               param_grid,
                               cv=5,
                               return_train_score=True)
    grid_search.fit(X_train,y_train)
    
    manager_salary_probabilities = pd.Series(y_hold_out_predicted[:,1], index = hold_out_data.index)
        
    return manager_salary_probabilities

salary_predictions()

1       0.126730
2       0.574744
5       0.959550
8       0.127498
14      0.310516
          ...   
992     0.008173
994     0.009198
996     0.008452
1000    0.048185
1001    0.128548
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [47]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(1)

FileNotFoundError: [Errno 2] File b'Future_Connections.csv' does not exist: b'Future_Connections.csv'

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

def new_connections_predictions():
    
    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G,future_connections.index)]
    future_connections['common neighbors'] = future_connections.index.map(lambda node: len(list(nx.common_neighbors(G,node[0],node[1]))))
    future_connections['common neighbors'] = future_connections['common neighbors']**3
    
    hold_out_data = future_connections.loc[future_connections['Future Connection'].isnull()]
    training_data = future_connections.loc[~future_connections['Future Connection'].isnull()]
    
    X_train, X_test, y_train, y_test = train_test_split(training_data[['preferential attachment','common neighbors']],
                                                                        training_data['Future Connection'],
                                                                        test_size = 0.25,
                                                                        random_state = 2)
    clf = LogisticRegression(C=.01,penalty='l2').fit(X_train,y_train)
    y_proba = clf.decision_function(X_test)
    
    #param_grid = {'C':[0.001,0.01,0.1,1,10,100],'penalty':['l1','l2']}
    #grid_search = GridSearchCV(LogisticRegression(),
                               #param_grid,
                               #cv=5,
                               #return_train_score=True)
    
    #grid_search.fit(X_train,y_train)
    
    hold_out_data_predicted = clf.predict_proba(hold_out_data[['preferential attachment','common neighbors']])
    future_connection_probability = pd.Series(hold_out_data_predicted[:,1], index = hold_out_data.index)
  
    return future_connection_probability

new_connections_predictions()

(107, 348)    0.034752
(542, 751)    0.033685
(20, 426)     0.654707
(50, 989)     0.033684
(942, 986)    0.033683
(324, 857)    0.033684
(13, 710)     0.076411
(19, 271)     0.076456
(319, 878)    0.033684
(659, 707)    0.033685
(49, 843)     0.033684
(208, 893)    0.033685
(377, 469)    0.033721
(405, 999)    0.033815
(129, 740)    0.033823
(292, 618)    0.033822
(239, 689)    0.033684
(359, 373)    0.033706
(53, 523)     0.034750
(276, 984)    0.033684
(202, 997)    0.033684
(604, 619)    0.034740
(270, 911)    0.033684
(261, 481)    0.043114
(200, 450)    0.876657
(213, 634)    0.033685
(644, 735)    0.034741
(346, 553)    0.033687
(521, 738)    0.033692
(422, 953)    0.033824
                ...   
(672, 848)    0.033684
(28, 127)     0.999502
(202, 661)    0.033688
(54, 195)     1.000000
(295, 864)    0.033685
(814, 936)    0.033685
(839, 874)    0.033683
(139, 843)    0.033684
(461, 544)    0.033694
(68, 487)     0.033695
(622, 932)    0.033685
(504, 936)    0.033831
(479, 528) 